#### ingest the nested drivers json file

- read json file
- rename columns and add new columns
- drop unwanted columns
- write output to silver container

In [0]:
from pyspark.sql.functions import current_timestamp, col, concat, lit
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DateType

In [0]:
name_schema = StructType([
    StructField("forename", StringType(), True),
    StructField("surname", StringType(), True)
])

In [0]:
drivers_schema = StructType([
    StructField("driverId", IntegerType(), False),
    StructField("driverRef", StringType(), True),
    StructField("number", IntegerType(), True),
    StructField("code", StringType(), True),
    StructField("url", StringType(), True),
    StructField("name", name_schema),
    StructField("dob", DateType(), True),
    StructField("nationality", StringType(), True),
])

In [0]:
drivers_df = spark.read \
    .schema(drivers_schema)\
    .json("abfss://bronze@motorracingdl.dfs.core.windows.net/drivers.json")

##### rename columns and add columns

In [0]:
drivers_with_columns_df = drivers_df.withColumnRenamed("driverId", "driver_id")\
                                    .withColumnRenamed("driverRef", "driver_ref")\
                                    .withColumn("ingestion_date", current_timestamp())\
                                    .withColumn("name", concat(col("name.forename"), lit(" "), col("name.surname")))


##### drop column url

In [0]:
drivers_final_df = drivers_with_columns_df.drop(col("url"))

In [0]:
# drivers_final_df.write.mode("overwrite").parquet("abfss://bronze@motorracingdl.dfs.core.windows.net/")